# Modeling residuals on university rankings

We'll try to apply the principles we saw in the diamonds-example to [this](https://www.kaggle.com/datasets/alitaqi000/world-university-rankings-2023) dataset. And not to make it a competition or anything, but let's see who has the most influence on the ranking of a university: the rate of international students or the teaching score!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv("files/World University Rankings 2023.csv")
df.head()

For the purpose of this exercise, we'll only be looking at countries with a lot of universities in them. Sort by descending amount of universities and keep only those rows from the top 10 countries. Show the value counts for these locations.

In [ ]:
# Identify the top 10 most frequent locations
top_10_locations = df['Location'].value_counts().nlargest(10).index

# Filter the dataframe to keep only rows with a location in the top 10
df = df[df['Location'].isin(top_10_locations)]
df.Location.value_counts()

Now do a boxplot for "Teaching score", our goal metric, split by country.

In [ ]:
sns.boxplot(data=df, x='Location', y='OverAll Score')

Not looking very nice, is it? That's because no data cleaning has been done yet and number of our metrics are numbers that are still stored as objects. Compare which ones by looking at the data and the datatypes.

In [ ]:
# DELETE

print(df.head())
print(df.dtypes)

An overview:
- Name of University             , object: Ok!
- Location                       , object: Ok!
- No of student                  , object: Wrong
- No of student per staff       , float64: Ok!
- International Student          , object: Wrong
- Female:Male Ratio              , object: Wrong
- OverAll Score                  , object: Wrong
- Teaching Score                , float64: Ok!
- Research Score                , float64: Ok!
- Citations Score               , float64: Ok!
- Industry Income Score         , float64: Ok!
- International Outlook Score   , float64: Ok!

Fix the wrong ones. Do you also feel the [lambda](https://www.geeksforgeeks.org/applying-lambda-functions-to-pandas-dataframe/)-vibe in this one?

(Challenges: nan-values, OverAll score sometimes contains "10.5-18.3" or something like that and the "-" isn't a normal "-", ratio has to be split and divided female/male, but there is an all female-college.)

In [35]:
# DELETE

def ratio_to_nr(ratio):
    if isinstance(ratio, float) or ratio.find(":") == -1:
        return 0
    data = str(ratio).split(" : ")
    if data[1] == "0":
        return 1
    return int(data[0])/int(data[1])

def overall_score_fix(score):
    if isinstance(score, float):
        return score
    
    if score.find("–") == -1:
        return float(score)
    
    data = score.split("–")
    data = [ float(x) for x in data]
    return (data[0] + data[1])/2

def percentage_to_float(percentage):
    if percentage == "%":
        return 0
    if isinstance(percentage, float):
        return percentage
    return float(percentage.replace("%",""))
    
df["No of student"] = df["No of student"].str.replace(",","").astype(int)
df["International Student"] = df["International Student"].apply(percentage_to_float)
df["Female:Male Ratio"] = df["Female:Male Ratio"].apply(ratio_to_nr)
df["OverAll Score"] = df["OverAll Score"].apply(overall_score_fix)

The above code-block can only be run once, since it stores the cleaned values in the fields itself.

But we were doing a boxplot! Note the rotation of the labels...

In [ ]:
ax = sns.boxplot(data=df, x='Location', y='OverAll Score')
dummy = plt.setp(ax.get_xticklabels(), rotation=45)

There seems to be some influence. What other metrics are important? Try a correlation matrix.

In [ ]:
# DELETE

# plot a correlation matrix of df
corr = df.corr(numeric_only=True)
dummy = sns.heatmap(corr, annot=True)

We are trying to maximize the OverAll Score, so if we look at that line and do some creative copy-pasting in paint, we get the following image.

![](files/2023-09-14-15-24-10.png)


Now analyze this data. What is the influence of the different columns?

(The following is a code-block, but we expect only text, the sort of which you'll also be expected to be able to produce on the exam.)

In [16]:
# DELETE

# An overview:
# - No of student: 0,069, small positive correlation, can be neglected
# - No of student per staff: -0.24, reasonable negative correlation. Negative so more staff per student means a better university
# - International Student: 0,59, good positive correlation
# - Female:Male Ratio: 0,096, small positive correlation, can be neglected. Note: higher number means more female students, so more female students mean a better university, although not statistically significant
# - OverAll Score: 1, because the same value
# - Teaching Score: 0,85, high positive correlation
# - Research Score: 0,88, high positive correlation
# - Citations Score: 0,85, high positive correlation
# - Industry Income Score: 0,37 reasonable positive correlation
# - International Outlook Score: 0,65, good positive correlation

# concluding: the influence of teaching score, research score and citations score is very big. These should be modelled first.

Also, copy-pasting in paint? Really? Show all values from the correlation matrix for OverAll Score order from high to low.

In [ ]:
# DELETE

corr["OverAll Score"].sort_values(ascending=False)


Let's look at the teaching score vs the overall score in a scatter plot. Add the percentage of international students as a color.

In [ ]:
# DELETE

# Draw a scatter plot of overall score vs teaching score. Add international student as color.

# plt.scatter(df["OverAll Score"], df["Teaching Score"])
sns.scatterplot(data=df, x="OverAll Score", y="Teaching Score", hue="International Student")


The dark dots (high number of international students) occur more on the right side of the plot, but some also appear on the left. We know there is a positive effect, but how big is it? To understand that we have to remove the influence of the Teaching score.

We'll assume the influence of teaching score on overall score is linear. No need to do a log this time.

Also, to make the linear regression model work we'll need to delete all na-values. Wish we would have done that before converting al the datatypes, it would have made our lives a lot easier.

In [ ]:
# DELETE

# print(df.count())
df.dropna(inplace=True)
# print(df.count())

from sklearn import datasets, linear_model

x = df["Teaching Score"].values.reshape(-1, 1)
y = df["OverAll Score"].values.reshape(-1, 1)

regr = linear_model.LinearRegression()
model = regr.fit(x, y)

fig, ax = plt.subplots(figsize=(12,6))

df.plot(kind='scatter', x="Teaching Score", y="OverAll Score", grid=True,fontsize=10, ax=ax,  figsize=(12, 6), alpha=0.1)
plt.plot(x, regr.predict(x), color='red')

Maybe print the parameters for the regression-line?

In [ ]:
# DELETE

print(f"a= {model.coef_[0][0]}, b= {model.intercept_[0]}")

And there is our linear model. Now we use this to predict the values for every weight. Once we have this predicted weight, we use it to calculate the residuals (or the error) for every actual value.

In [ ]:
# DELETE

df['overall_score_predicted'] = model.predict(df["Teaching Score"].values.reshape(-1, 1))
df['overall_score_residuals'] = df["OverAll Score"] - df['overall_score_predicted']

# df.head()
df.plot(kind='scatter', x="Teaching Score", y="overall_score_residuals", grid=True,fontsize=10, figsize=(12, 6), alpha=0.5)

What we have here is a plot of teaching score vs the residuals of the overall score. These residuals imply that the teaching score has no predictive value anymore and that is actually the case: the data is almost random.

Well, not quite. There are still some lines left because the research score and the citations score are still in there. But if we plot the residuals of the the overall score vs the international students, what do we get?

In [ ]:
# DELETE

df.plot(kind='scatter', y="International Student", x="overall_score_residuals", grid=True,fontsize=10, figsize=(12, 6), alpha=0.5)

We see that:
- Most schools have a low percentage of international students (< 20). They have both positive and negative residuals, so both bad and good scores.
- Above 20% international students we don't see many below 0 residuals anymore. This means that there is indeed a trend that more international students mean better schools.

But was all this necessary? Couldn't we have just made the same plot with the original data?

In [ ]:
# DELETE

df.plot(kind='scatter', y="International Student", x="OverAll Score", grid=True,fontsize=10, figsize=(12, 6), alpha=0.5)

The trend is indeed there, but less explicit than after applying the model. This should become even clearer when you remove the research and the citations score from the residuals.